In [ ]:
# This notebook extract the haptic convnet features, and save them into a pickle file
# To get started, you should set the following configurations and then run all cells
# configuration starts
task="conv" # the usual convolutional features, output feature pickle file
#task="lstm"  # the lstm features, directly test the AUC score

#task="leave1out" # the features that predict left out objects, output the feature pickle file 
# Note that in the task of "leave1out", you should manually replace "data_conv" with "data_leave1out" 
#     in ../protos/concat2.yangnet.train

lstmPycaffePath='/home/yang/caffe-lstm-201509/python'
convPycaffePath='/home/yang/caffe-201509/caffe/python'
hapticDataPath="../../0_get_data/data/data_"+task+"/"
protoPath='./protos'
if task=="conv":
    hapticFeatureOutputDir='haptic_features.pkl.rand17'
else:
    hapticFeatureOutputDir='haptic_features.pkl.'+task
# configuration ends



# setting related flags and files
# set the useLstm flag
if task=="lstm":
    useLstm=True
else:
    useLstm=False 
    
# get the right caffe module, and cp the right run script
import sys
if useLstm:
    sys.path.insert(0, lstmPycaffePath)
    ! rm run_haptic_roc.sh
    ! cp run_haptic_roc_lstm.sh run_haptic_roc.sh
else:
    sys.path.insert(0, convPycaffePath)
    ! rm run_haptic_roc.sh
    ! cp run_haptic_roc_conv.sh run_haptic_roc.sh

# change the path
if not ('hasChanged' in dir()):
    import os
    hasChanged=True
    os.chdir(protoPath)
curDir=!pwd
print "current dir: ", curDir    

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import caffe
print "Caffe path:", caffe.__file__
from sklearn.metrics import roc_curve, auc
print "import ok"


# some helper functions
def pprint(l):
    for ele in l:
        print ele

def vargmax(blob):
    ans=np.zeros((blob.shape[0], 1));
    for i in range(len(ans)):
        if blob[i][0]==blob[i][1]:
            ans[i]=1
        else:
            ans[i]=blob[i].argmax();
    return ans

def accuracy(pre, label):
    pre=pre.reshape(len(pre))
    label=label.reshape(len(label))
    return (1.0*(pre==label).sum()/len(pre))

def f1score(pre, label):
    pre=pre.reshape(len(pre))
    label=label.reshape(len(label))
    
    tp=sum(pre*label)
    fp=sum(pre*(1-label))
    fn=sum((1-pre)*label)
    return (2.0*tp)/(2*tp+fp+fn+1e-10)

# From Num*2 to score
def toScore(ip2):
    return ip2[:,1]-ip2[:,0]

# From score and label to AUC (with ROC figure)
def roc_sample(score, label, showFigure=True):
    fpr, tpr, _ = roc_curve(label, score)
    roc_auc = auc(fpr, tpr)
    
    if showFigure:
        # Plot of a ROC curve for a specific class
        plt.figure()
        plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic')
        plt.legend(loc="lower right")
        plt.show()
    return roc_auc


def changeTestProto(gTestProto, sz):
    ! cat $gTestProto | sed "s\input_dim: xxx\input_dim: {sz}\\" > tmp
    ! rm $gTestProto
    ! mv tmp $gTestProto

def changeTestProtoBack(gTestProto, sz):
    ! cat $gTestProto | sed "s\input_dim: {sz}\input_dim: xxx\\" > tmp
    ! rm $gTestProto
    ! mv tmp $gTestProto

import h5py    
def testOn(testDataPath, protoFilePath, select):
    # import the data and test using FORWARD
    global varList
    beforeFirst=True
    with h5py.File(testDataPath, 'r') as f:
        for var in varList:
            tmp=f[var][:]
            sp=tmp.shape
            
            # first time setup the network
            if beforeFirst:
                beforeFirst=False
                if useLstm:
                    changeTestProto(gTestProto, sp[0])
                    net = caffe.Net(protoFilePath, './snapshot_yang/_iter_'+str(select*10)+'.caffemodel')        
                else:
                    net = caffe.Net(protoFilePath, './snapshot_yang/_iter_'+str(select*10)+'.caffemodel', caffe.TRAIN)

        
            net.blobs[var].reshape(sp[0],sp[1],sp[2],sp[3])
            net.blobs[var].data[...]=tmp
            
    net.forward()
    
    if useLstm:
        changeTestProtoBack(gTestProto, sp[0])
    return net


# Read model from saved model file, run through the net with (validate/test) data
# return ip2 and label
def getIp2Label(select, phase):
    global gValidateProto, gTestProto
    fun=lambda fname: open(fname,'r').readline().strip()
    if phase=="train":
        net=testOn(fun(hapticDataPath+'haptic_validate.txt'), gValidateProto, select)
    elif phase=="test":
        net=testOn(fun(hapticDataPath+'haptic_test.txt'), gTestProto, select)
    else:
        print "phase is neither train or test"
        
    ip2=net.blobs['ip2'].data
    label=net.blobs['label'].data
    return (ip2.reshape((ip2.shape[0],2)), label.reshape((label.shape[0],1)))

# select model by maximizing the AUC of validation file
def selectAUC(accs):
    bestAuc=0;
    bestId=1;
    for i in range(1, len(accs)):
        ip2, label=getIp2Label(i, "train")
        nowAuc=roc_sample(toScore(ip2), label, False)
        if nowAuc>bestAuc:
            bestAuc=nowAuc
            bestId=i
    return bestId

def selectAccuracy(accs):
    # select the max based on accuracy
    return np.argmax(accs)

# new for voting
def voteScore(ip2):
    global enableTwoFin
    if enableTwoFin:
        cnt=10
    else:
        cnt=5
    pre=vargmax(ip2);
    N=pre.shape[0];
    N5=N/cnt;
    ansScore=np.zeros((N5, 1))
    ansLabel=np.zeros((N5, 1))
    for i in range(N5):
        acc=0
        for j in range(cnt):
            acc += pre[i*cnt+j];
        ansScore[i]=acc;
        if acc>(cnt/2):
            ansLabel[i]=1
        else:
            ansLabel[i]=0
    return (ansScore, ansLabel)

def voteLabel(label):
    global enableTwoFin
    if enableTwoFin:
        step=10
    else:
        step=5
    N=label.shape[0];
    for i in range(step-1):
        assert label[range(i, N, step)].sum()==label[range(i+1, N, step)].sum()
    return label[range(0, N, step)]

def getTestAUC(select):
    global enableTwoFin, enableVoting
    ip2, label=getIp2Label(select, 'test')
    
    print "Test accuracy (NO voting): %f" % (accuracy(vargmax(ip2), label))
    if enableVoting:
        # add to support voting 
        vScore, vPre=voteScore(ip2);
        vLabel=voteLabel(label);
    else:
        vScore=toScore(ip2)
        vPre=vargmax(ip2);
        vLabel=label
        
    print "Test accuracy (Voting=%s, TwoFin=%s): %f" \
            % (enableVoting, enableTwoFin, accuracy(vLabel, vPre))
    try:
        roc=roc_sample(vScore, vLabel, True);
    except:
        print "Failed to calculate AUC on this testing dataset"
        roc=None
    return (roc, vScore, vPre, vLabel)

def changeAdj(iadj_str):
    prefix=hapticDataPath
    !echo $prefix{iadj_str}"train.h5" > $prefix"haptic_train.txt"
    !echo $prefix{iadj_str}"test.h5" > $prefix"haptic_test.txt"
    !echo $prefix{iadj_str}"validate.h5" > $prefix"haptic_validate.txt"

def changeAdjNovalid(iadj_str):
    prefix=hapticDataPath
    !echo $prefix{iadj_str}"train.h5" > $prefix"haptic_train.txt"
    !echo $prefix{iadj_str}"test.h5" > $prefix"haptic_test.txt"
    if task=="conv" or task=="lstm":
        !echo $prefix{iadj_str}"validate.h5" >> $prefix"haptic_train.txt"
    # for backward compatibility
    !echo $prefix{iadj_str}"validate.h5" > $prefix"haptic_validate.txt"    
 
def getAdjName(i): 
    adjectives=['absorbent','bumpy','compressible','cool','crinkly','fuzzy','hairy','hard','metallic','nice','porous',
            'rough','scratchy','slippery','smooth','soft',
            'solid','springy','squishy','sticky','textured','thick','thin','unpleasant']
    # because adjectives has exclude adjective 7
    if i<7:
        adjName=adjectives[i-1]
    else:
        adjName=adjectives[i-2]
    return adjName


def oneAdj(iadj):  
    # debug, Yang, no validation
    #changeAdj(str(iadj))
    changeAdjNovalid(str(iadj))
    
    # call the training process, get the training result
    print "start training adjective %d, %s" % (iadj, getAdjName(iadj))
    res=! ./run_haptic_roc.sh
    
    # map the accuracy result to numbers (the last one is the time measurement, which is omitted)
    #accs=map(lambda str: float(str.split()[-1]), trainRes[0:-1])
    accs=[0]*60
    print "Validation accuracies are: ", accs

    try:
        # debug
        #select=selectAUC(accs)
        #select=selectAccuracy(accs)
        if useLstm:
            select=30
        else:
            select=19
        print "selecting the ", select*10, " iteration"
        #print "Model selection method: AUC"
    except:
        print "Model selection method: Accuracy (Due to error of AUC)"
        select=selectAccuracy(accs)
    print "\tBest iteration: %d\n \tValidation accuracy: %f"% (select*10, accs[select])

    testAUC, vScore, vPre, vLabel = getTestAUC(select);
    return (testAUC, vScore, vPre, vLabel)

def allAdjs(Total, start=1): 
    #pennExclusion=[7, 11, 21, 25];
    exclusion=[7];
    notEnoughToTrain=[2,5,7,8,10,11,21,24,25];
    
    accAll=[];
    f1All=[]
    aucAll=[]
    accEnough=[];
    global prefix
    import pickle
    outName=prefix.split('/')[-1]+"pkl"
    
    if os.path.isfile(outName):
        print "load previous results"
        preRes=pickle.load(open(outName, 'r'))
        accAll=preRes['accAll']
        f1All=preRes['f1All']
        aucAll=preRes['aucAll']
        accEnough=preRes['accEnough']

    scores=np.array([])
    labels=np.array([])
    index=0
    for i in range(1, Total+1):
        if i<start:
            print "skipping: ", i, " because start param"
            continue
        if exclusion.count(i)==0:
            index += 1
            if index<=len(accAll):
                print "skipping ", i, "th adjective"
                continue
            
            auc, score, pre, label=oneAdj(i)
            scores=np.concatenate(( scores, score.flatten() ))
            labels=np.concatenate(( labels, label.flatten() ))
            # add support for average accuracy
            accAll += [accuracy(pre, label)];
            f1All += [f1score(pre, label)];
            aucAll += [auc]
            print "Test F1-Score: %f, AUC score %f" % (f1All[-1], aucAll[-1])
            if notEnoughToTrain.count(i)==0:
                accEnough+=[accAll[-1]];
            pickle.dump({'accAll': accAll, 'f1All':f1All, 'aucAll':aucAll, 'accEnough':accEnough }, open(outName,'w'))
            
    changeAdj("")
    #roc_sample(scores.ravel(), labels.ravel(), True)
    
    print "average accuracy is %f out of (%d)" %(np.mean(accAll), len(accAll))
    print "enough accuracy is %f out of (%d)" %(np.mean(accEnough), len(accEnough))
    print "average f1-socre is %f out of (%d)" %(np.mean(f1All), len(f1All))
    print "average AUC socre is %f out of (%d)" %(np.mean(aucAll), len(aucAll))

def changeRunScript(gTrainProto):
    ! cat run_haptic_roc.sh | sed "s\trainProtoName=.*\trainProtoName='{gTrainProto}'\\" > tmp
    ! rm run_haptic_roc.sh
    ! mv tmp run_haptic_roc.sh
    ! chmod +x run_haptic_roc.sh

global gTrainProto, gValidateProto, gTestProto
def setGlobals(prefix):
    global gTrainProto, gValidateProto, gTestProto
    gTrainProto=prefix+"yangnet.train"
    gValidateProto=prefix+"yangnet.validate"
    gTestProto=prefix+"yangnet.test"
    # change run_haptic_roc
    changeRunScript(gTrainProto)
    
global enableVoting, enableTwoFin, varList, prefix
enableVoting=False
enableTwoFin=False

#prefix="./first."
#varList=["ep1","ep2","ep3","ep4","label"]

#prefix="./epca2."
#varList=["ep1","ep2","ep3","ep4","epca1","epca2","epca3","epca4","label"]

if not useLstm:
    prefix="./concat2."
    varList=["concat","label"]
else:
    prefix="./lstm."
    varList=["cont", "lstm", "label"]

setGlobals(prefix)
# fields in proto files: snapshot, test iter, batch size, source
# each proto file must contain "ip2" and "label"

In [ ]:
if task=="conv" or task=="leave1out":
    if task=="leave1out":
        useLeave1out=True # this means omitting the validation files
    else:
        useLeave1out=False

    # extract the training and testing features
    # output structure: ans[iadj]['train'/'test']=np.zeros((instances, 16))
    global gValidateProto
    featureName='conv3' # for concat2
    #featureName='ep_ip' # for epca2

    hapFs=[None]*25
    for iadj in range(1,26):
        if iadj==7:
            continue
            # skipping the adjective

        i=iadj-1
        hapFs[i]={}

        # train the corresponding network
        changeAdjNovalid(str(iadj))
        print "start training adjective %d, %s" % (iadj, getAdjName(iadj))
        trainRes=! ./run_haptic_roc.sh

        print "\t extracting features"
        # extract the features
        fun=lambda fname: open(fname,'r').readlines()
        for phase in ['train', 'test']:
            hapFs[i][phase]={}
            lines=fun(hapticDataPath+'haptic_'+phase+'.txt');
            for line in lines:
                print "phase=", phase, "h5file=", line.strip()
                if useLeave1out and line.strip().endswith("validate.h5"):
                    continue
                net=testOn(line.strip(), gValidateProto, 19)
                if hapFs[i][phase].has_key('feature'):
                    hapFs[i][phase]['feature']=np.concatenate((hapFs[i][phase]['feature'], 
                                                               np.squeeze(net.blobs[featureName].data) ))# num*128*5
                    hapFs[i][phase]['label']=np.concatenate((hapFs[i][phase]['label'], 
                                                             np.squeeze(net.blobs['label'].data) ))#num
                else:
                    hapFs[i][phase]['feature']=np.squeeze(net.blobs[featureName].data) # num*128*5
                    hapFs[i][phase]['label']=np.squeeze(net.blobs['label'].data) #num

    changeAdj("")

    import pickle
    pickle.dump(hapFs, open(hapticFeatureOutputDir,'w'))
    print "features saved"
elif task=="lstm":
    allAdjs(25)
else:
    print "unknown task"